# Introduction

**Incarceration & COVID-19: How Jails Respond to COVID**<br>

This project scrapes daily jail roster information to create a large dataset. This dataset is designed to analyze how jail populations have fluctuated in response to COVID-19. Research centers on explaining why county jails in different parts of the United States have responded differently to the pandemic over time. 

A separate but related idea for this dataset analyzes the impact of pandemic-related jail population declines on local crime. This project uses daily jail roster population counts as the focal variable and the analysis uses group-based trajectory modeling. Our scraped data will address gaps in the [Vera](https://github.com/vera-institute/jail-population-data) dataset.

We start by comparing Washington and New York states because they dealt with COVID-19 at the early on-set of the pandemic. Below is a list of the data points to collect to harmonize with the Vera data.
- County Name
- State Name
- Daily Population Counts
- Reporting Jail Name

# Imports

In [1]:
# Import standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from datetime import datetime

# API libraries
import re
import os
from os import system  
import time
from time import sleep
import json
import random
import requests
# from math import floor
# from copy import deepcopy

# Selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#SQL
import sqlite3
import mysql.connector

# MySQL

In [10]:
# FIGURE OUT PASSWORD STUFF

mydb = mysql.connector.connect(host='localhost',\
                              user='root',\
                              passwd='Smoke4fire',\
                              database='testdb')
print(mydb)

ProgrammingError: 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES)

In [9]:
# Created a db

mycursor = mydb.cursor()
# mycursor.execute("CREATE DATABASE testdb")

NameError: name 'mydb' is not defined

In [4]:
mycursor.execute("SHOW DATABASES")

for db in mycursor:
    print(db)

('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('testdb',)


In [6]:
#Created a table

# mycursor.execute("CREATE TABLE jails\
#                  (reporting_jurisdictions VARCHAR(100),\
#                  county_name VARCHAR(100),\
#                  state_name VARCHAR(100),\
#                  Date VARCHAR(100),\
#                  jail_population INTEGER(255))")

In [7]:
mycursor.execute("SHOW TABLES")

for tb in mycursor:
    print(tb)

('jails',)


In [11]:
sqlFormula = "INSERT INTO jails (Date, reporting_jurisdictions, county_name, \
state_name, jail_population) VALUES (%s, %s, %s, %s, %s)"

In [13]:
#USE FOR ALL COMMITS
# mycursor.execute(sqlFormula, jail1)
# mydb.commit()

# States

Be sure to check for APIs in addition to scraping. Will include NY, WA and FL.

## Washington

### Whitman

In [11]:
url = "http://www.whitmancountyjail.org/"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
driver.implicitly_wait(3)
driver.get(url)

In [13]:
listy = driver.find_elements_by_css_selector('h4')

#view full list
# for x in listy[:50]:
#     if len(x.text) > 0:
#         print(x.text)

In [12]:
location = driver.find_element_by_xpath('//*[@id="form1"]/footer').text
location

"Whitman County Jail, 411 N. Mill Street, Colfax WA 99111.\nThis Website was created and is maintained by Lobo Public Safety Software.\nContact: web@lobopublicsafetysoftware.com.\nThis site is served by Lobo Webhosting.\n\nWhitman County Jail - Inmate Listing Website v1.1.5.0524 Copyright © 2020\nWhitman County Sheriff's Office | Disclaimer"

In [ ]:
JWhitman = (location[:19])
CWhitman = (location[:7])
SWhitman = (location[47:49])
DWhitman = datetime.now().strftime('%Y-%m-%d')
PWhitman = (len(listy))-10

print('reporting_jurisdictions = ',JWhitman)
print('county_name = ',CWhitman)
print('state_name = ',SWhitman)
print('Date = ',DWhitman)
print('jail_population = ',PWhitman)

In [ ]:
driver.close()

### Spokane

#### Selenium

In [53]:
url = "https://www.spokanecounty.org/352/Inmate-Roster'"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
driver.implicitly_wait(3)
driver.get(url)

# print(driver.page_source)

In [54]:
Sinmate = driver.find_element_by_xpath('//*[@id="tblInmateRoster_info"]').text
print(Sinmate)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="tblInmateRoster_info"]"}
  (Session info: chrome=83.0.4103.97)


In [55]:
driver.close()

### Okanogan

Details can be found in the Daily Jail Inmate Log on [Okanogan Sherriff Website](https://okanogansheriff.org/).

In [65]:
url = "https://okanogansheriff.org/"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
driver.implicitly_wait(3)
driver.get(url)

print(driver.page_source)

<html><head>
<meta http-equiv="Content-Type" content="text/html">
<meta name="description" content="The web site for the Okanogan County Sheriff's Office. Welcome to the new Okanogan County Sheriff's Office web site! Our web site is designed to be another resource linking the community to the Okanogan County Sheriff's Office, making it easier for you to locate the information, resources and services you need, ranging from public safety and education to community outreach activities.  We will continue to update and enhance the Okanogan County Sheriff's Office web site frequently, providing more services and accessibility to all..">
<meta name="keywords" content="Sheriff, Sheriff's Office, Okanogan County, Washington, Omak, Oroville, Tonasket, Brewster, Twisp, Winthrop, Methow, county, law enforcement, policing, northwest, eastern washington, okanogan river, similkameen, officers, patrol, deputy, deputies, news releases, most wanted, lakes, volunteers, employment, corrections, investigat

In [64]:
# Need to read from a pdf name=CD6F9816E7949144F43AB92B6CCADAA8

location = driver.find_element_by_xpath('/html/frameset/frame[2]').text
location

''

In [66]:
driver.close()

### Jefferson

[Jefferson](https://co.jefferson.wa.us/174/Jail-Inmate-Search)<br> To view the full inmate roster click the Clear button then the Search button.

In [58]:
url = "https://co.jefferson.wa.us/174/Jail-Inmate-Search"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
driver.implicitly_wait(3)
driver.get(url)

# print(driver.page_source)

In [59]:
#Hidden input type

inmate = driver.find_elements_by_name('Name')
print(len(inmate))

0


In [60]:
driver.close()

### Grant

[Grant](ttps://www.grantcountywa.gov/SHERIFF/Corrections/Inmate-Roster.htm), daily pdf

In [70]:
url = "https://www.grantcountywa.gov/SHERIFF/Corrections/Roster-InmateinmateRoster%20v%206.rpt.pdf"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
driver.implicitly_wait(3)
driver.get(url)

print(driver.page_source)

<html><head></head><body style="height: 100%; width: 100%; overflow: hidden; margin:0px; background-color: rgb(82, 86, 89);"><embed name="97990C66DAA194EA3D193266385FD0FD" style="position:absolute; left: 0; top: 0;" width="100%" height="100%" src="about:blank" type="application/pdf" internalid="97990C66DAA194EA3D193266385FD0FD"></body></html>


In [71]:
driver.close()

### Gray's Harbor

[Gray's Harbor]('http://ghlea.com/JailRosters/GHCJRoster.html')

In [ ]:
url = "http://ghlea.com/JailRosters/GHCJRoster.html"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
driver.implicitly_wait(3)
driver.get(url)

# print(driver.page_source)

In [ ]:
inmate = driver.find_elements_by_xpath('//*[@id="main-table"]/tbody/tr')
print(len(inmate))

In [ ]:
driver.close()

### Ferry

[Ferry](https://www.ferry-county.com/Courts%20and%20Law/Inmate%20Roster/Inmate_Roster_Page.html): in the section that says "MAY 11, 2020 - 8 inmates")

In [ ]:
url = "https://www.ferry-county.com/Courts%20and%20Law/Inmate%20Roster/Inmate_Roster_Page.html"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
driver.implicitly_wait(3)
driver.get(url)

# print(driver.page_source)

In [ ]:
Finmate = driver.find_element_by_xpath('//*[@id="mainContent3"]/p[9]').text
Finmate[16]

In [ ]:
driver.close()

### Clallam

[Clallam](https://websrv23.clallam.net/NewWorld.InmateInquiry/WA0050000/)

In [14]:
url = "https://websrv23.clallam.net/NewWorld.InmateInquiry/WA0050000/"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver') 
driver.implicitly_wait(3)
driver.get(url)

# print(driver.page_source)

In [22]:
Clallam_inmate = driver.find_elements_by_class_name('Name')
print(len(Clallam_inmate))

77


In [23]:
driver.close()

### Adams

[View](https://www.co.adams.wa.us/government/jail_roster_and_booking_information/index.php) Jail Roster Information

In [45]:
url = "https://www.co.adams.wa.us/jailrosterout.txt"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
driver.implicitly_wait(3)
driver.get(url)

# print(driver.page_source)

In [46]:
Adams_text=driver.find_element_by_xpath('/html/body/pre').text
Adams_text.count("Booking")

20

In [47]:
driver.close()